## Структура решающих деревьев

Решающие деревья применяются в основном в задачах классификации и регрессии в [машинном обучении]($Введение в машинное обучение$) на табличных данных. В общем виде решающее дерево - это иерархическая схема принятия решений в виде графа. В промежуточных вершинах (звеньях) проверяются некие условия, и в зависимости от результатов выбирается путь в графе, который приводит к одной из конечных вершин (листьев).

В общем случае звенья и листья могут быть сложными функциями. Однако в большинстве практических реализаций (scikit-learn, XGBoost, CatBoost **???**) каждый лист соответстствует константному ответу, а в каждом звене проверяется значение лишь одного признака (*рис. 1*). Если это количественный признак, то все значения больше некоего порога отправляются по одному пути, меньше порога - по другому пути. Если это категориальный признак, то одна или несколько категорий отправляется по одному пути, остальные категории по другому пути (*категориальные признаки иногда преобразуются в количественные, более подробно рассмотрим позже*).

<img src="assets/tree.png" width="800" align="center">

<center><i>Рис. 1. Пример решающего дерева для задачи бинарной классификации (<a href="https://www.researchgate.net/figure/Decision-tree-scheme-for-the-samples-and-cutting-values-of-subsets_fig3_334061532">источник</a>). Принятие решения начинается с верхней вершины графа (корня).</i></center>

С первого взгляда кажется, что такой подход хорошо подходит для одних задач, но очень плохо подходит для других задач.

**Позитивный пример.** Многие способы принятия решений выглядят как блок-схема. Например при постановке диагноза врач сначала может измерить температуру. Если температура выше определенного порога, то может заподозрить простуду и посмотреть горло, послушать легкие и так далее. Листьями решающего дерева окажутся конкретные диагнозы. В целом, решающее дерево хорошо следует идее о том, что на некоторые признаки нужно обращать внимание только при условии определенных значений других признаков.

**Негативный пример.** Попробуем аппроксимировать функцию $y(x) = x_2 - x_1$ (*рис. 2a*) решающим деревом (*рис. 2b*). Решающее дерево разбивает все пространство признаков на прямоугольные области, в каждой из которых ответом является константа. В данной задаче такой подход явно неоптимален. Во-первых, если решающее дерево представить в виде функции $f(x)$, то эта функция будет разрывной (кусочно-постоянной), тогда как целевая функция $y(x)$ непрерывна. Во-вторых, на ответ влияет разность $x_2 - x_1$, а в каждом звене дерева проверяется либо $x_1$, либо $x_2$, что тоже неоптимально. Для достижения хорошей точности потребуется большое и сложное дерево, хотя сама задача определения $y$ очень проста. И наконец, поскольку решающее дерево обучается на данных (о способе обучения мы поговорим далее), то в тех областях, где мало данных или они вовсе отсутствуют, функция $f(x)$ будет продолжена неправильно (проблема экстраполяции).

<img src="assets/trees1.png" width="1000" align="center">

<center><i>Рис. 2. Приближение непрерывной функции одним решающим деревом и ансамблем решающих деревьев.</i></center>

Проблему разрывности функции $f(x)$ и негладкости границ между листьями можно смягчить, если обучать не одно решающее дерево, а много деревьев, и усреднять их ответы (*рис. 2c*). При этом деревья либо должны обучаться на разных подвыборках данных, либо сам процесс обучения должен содержать элемент случайности, чтобы деревья получились неодинаковыми. Поэтому на практике обучают так называемый "случайный лес" из множества деревьев с помощью алгоритмов бэггинга или бустинга, которые мы расмотрим позже. Проблема экстраполяции при этом все же остается - например, случайный лес не может предсказывать значения, большие или меньшие всех тех, что встречались при обучении.

На нашей модельной задаче (*рис. 2*) решающие деревья дают не впечатляющий результат, тогда как, например, нейронная сеть в этой же задаче обучается почти точно аппроксимировать целевую функцию. Но на реальных задачах, наоборот, ансамбли решающих деревьев сейчас являются одними из самых эффективных моделей.

## Обучение решающих деревьев

Допустим мы имеем обучающую выборку из пар $\{x_i, y_i\}_{i=1}^N$ и хотим построить по нему решающее дерево. Можно без труда построить сколько угодно деревьев, дающих на обучающей выборке 100%-ю точность (если в ней нет примеров с одинаковыми $x$, но разными $y$). Для этого достаточно в каждой вершине выбирать для разделения любой признак и любой порог, и тогда рано или поздно все примеры попадут в разные листья. Но нам важна не точность на обучающей выборке, а степень обобщения - то есть точность на всем порождающем распределении, из которого взята обучающая выборка.

Можно воспользоваться принципом бритвы Оккама, который говорит, что простые гипотезы предпочтительнее сложных, и попробовать построить как можно более простое дерево. Однако задача нахождения наиболее простого решающего дерева для данного датасета (по суммрному количеству листьев или по средней длине пути в графе) является NP-полной ([Hancock et al., 1996]($Lower Bounds on Learning Decision Lists and Trees$), [Hyafil and Rivest, 1976]($Constructing optimal binary decision trees is NP-complete$)), то есть (если $P \neq NP$) экспоненциально сложной.

Вообще многие задачи являются экспоненциально сложными, если искать лучшее решение из всех возможных, то есть выполнять исчерпывающий поиск (exhaustive search). Но это не мешает находить хорошие приближенные решения, выполняя либо жадный поиск (greedy search), либо лучевой поиск (beam search). Жадный поиск означает, что на каждом шаге мы ищем локально оптимальное решение, то есть решение, приводящее к наибольшему "сиюминутному" выигрышу. Например, знакомясь с девушкой, которая вам нравится, вы не можете перебрать все возможные варианты развития диалога и заранее выбрать наилучший (т. е. выполнить исчерпывающий поиск), но можете после каждой фразы искать оптимальное продолжение диалога (жадный поиск).

*Примечание: пока работаем с количественными признаками, работу с категориальными признаками рассмотрим позднее.*

### Построение решающего дерева

Применительно к решающим деревьям жадный поиск означает, что мы строим дерево пошагово, на каждом шаге заменяя один из листьев на разделяющее правило, ведущее к двум листьям. На каждом шаге мы должны выбрать:

1. Лист $L$, который заменяем решающим правилом
2. Способ разделения (признак и порог)
3. Значения $C$ и $C^\prime$ на двух новых листьях

Этот выбор мы должны сделать так, чтобы функция потерь всего дерева на обучающей выборке уменьшилась как можно сильнее. Для этого достаточно рассмотреть функцию потерь на примерах из обучающей выборки, которые относятся к листу $L$.

Шаг алгоритма выглядит следующим образом: мы перебираем все возможные листья, признаки и пороги (если обучающая выборка имеет размер $N$, то каждый признак принимает не более $N$ различных значений, поэтому для каждого признака имеет смысл перебирать не более $N-1$ порогов). Для каждого листа, признака и порога определяем значения $C$ и $C^\prime$ на листьях и считаем функцию потерь получившегося дерева. В итоге мы выбираем тот лист, признак и порог, для которых значение функции потерь наименьшее, и дополняем дерево новым разделяющим правилом.

Осталось ответить на вопрос: как именно определяются значения $C$ и $C^\prime$ на двух новых листьях?

Пусть после добавления разделяющего правила в один лист попало $N_1$ примеров со значениями целевого признака $y_1, \dots, y_{N_1}$, в другой лист попало $N_2$ примеров со значениями целевого признака $y^\prime_1, \dots, y^\prime_{N_2}$. Запишем суммарную функцию потерь после разделения, и минимизируем ее по $C$ и $C^\prime$:

$\sum\limits_{i=1}^{N_1} {Loss}(y_i, C) + \sum\limits_{i=1}^{N_2} {Loss}(y^\prime_i, C^\prime) \to \underset{C, C^\prime}{\min} \tag{1}$

Первое слагаемое зависит только от $C$, второе только от $C^\prime$, поэтому их можно минимизировать независимо.

### Оптимальное разделение в задаче регрессии

В качестве функции потерь выберем среднеквадратичную ошибку. Исходя из формулы $(1)$, на первом листе нужно выбрать такое значение $C$ на первом листе, что:

$\sum\limits_{i=1}^{N_1} (C - y_i)^2 \to \underset{C}{\min}$

Поскольку $y_i$ - константы, то задача означает поиск минимума функции от одной переменной. Взяв производную и приравняв ее к нулю мы найдем, что $C$ - среднее арифметическое значений $y_i$. Аналогично, $C^\prime$ - среднее арифметическое значений $y_i^\prime$.

Если бы мы минимизировали не среднеквадратичную ошибку (MSE), а среднюю абсолютную ошибку (MAE), тогда оптимальным значением $C$ была бы медиана значений $y_i$.

### Оптимальное разделение в задаче классификации

Мы можем пойти стандартным путем: пусть каждый лист выдает вероятности классов, а в качестве функции потерь используем категориальную перекрестную энтропию (logloss). Пусть количество классов равно $K$. Введем следующие обозначения:

- $\{p_k\}_{i=1}^{K}$ - доля объектов $k$-го класса в первом листе
- $\{p_k^\prime\}_{i=1}^{K}$ - доля объектов $k$-го класса во втором листе
- $\{C_k\}_{i=1}^{K}$ - предсказываемая вероятность $k$-го класса в первом листе
- $\{C_k^\prime\}_{i=1}^{K}$ - предсказываемая вероятность $k$-го класса во втором листе

Минимизируем суммарную ошибку на первом листе. Количество примеров в первом листе, для которых верным ответом является класс $k$, равно $N_1 p_k$. Суммируем ошибку по всем классам и будем искать минимум по $C_1, \dots, C_K$:

$ - \sum\limits_{k=1}^{K} N_1 p_k \log C_k \to \underset{C_1, \dots, C_K}{\min} \qquad \sum\limits_{k=1}^{K} C_k = 1$

Из данной формулы можно исключить $N_1$, и тогда задача нахождения оптимального распределения $C_k$ сводится к минимизации перекрестной энтропии между распределениями $p_k$ и $C_k$. Это эквивалентно минимизации расхождения Кульбака-Лейблера между $p_k$ и $C_k$ и достигается при $p_k = C_k$ для всех $k$. Полученный результат интуитивно понятен: если, например,  в листе 10% примеров класса 0 и 90% примеров класса 1, то предсказывая ответ для неизвестного примера оптимальным вариантом будет назначить классам вероятности 10% и 90%.

Таким образом, мы нашли, что оптимальное $C_k$ равно $p_k$. Суммарная функция потерь на обоих листьях будет равна:

$ - N_1 \sum\limits_{k=1}^{K} p_k \log p_k - N_2 \sum\limits_{k=1}^{K} p_k^\prime \log p_k^\prime = N_1 H(p_k) + N_2 H(p_k^\prime)$

Здесь $H(p_k)$ - энтропия распределения $p_k$ (ее более корректно записывать как $H(\{p_k\}_{i=1}^{K})$). Иными словами, нам нужно искать такой признак и порог, чтобы минимизировать взвешенную сумму энтропий распределений классов в обоих листьях. Энтропию можно интерпретировать как степень неопределенности, или "загрязненности" (impurity) распределения, и энтропия достигает минимума в том случае, если распределение вероятностей назначает вероятность 1 одному из классов.

Минимизация взвешенной суммы энтропий называется энтропийным критерием разделения. Интересно, что если мы будем минимизировать не кроссэнтропию, а среднюю абсолютную ошибку (MAE) между предсказанным распределением $C_k$ и истинным (эмпирическим) распределением, которое назначает вероятность 1 верному классу, то придем к критерию Джини:

$N_1 \sum\limits_{k=1}^{K} p_k (1 - p_k) + N_2 \sum\limits_{k=1}^{K} p_k^\prime (1 - p_k^\prime) \to \min$

На практике чаще используется энтропийный критерий, потому что он соответствует перекрестной энтропии, которая чаще других применяется в задачах классификации.

### Критерий остановки и обрезка дерева

Мы рассмотрели шаг роста дерева, но остается еще один вопрос: до какой степени нужно растить дерево? На *рис. 3* показаны шаги роста дерева для классификации по энтропийному критерию. На первых шагах дерево находит явные закономерности в распределении классов, но затем начинает переобучаться на случайном шуме. Если мы соберем ансамбль из деревьев, то проблема переобучения ослабнет. Однако чем больше дерево, тем больше вычислительных ресурсов требуется для его обучения, поэтому размер дерева в ансамбле все же имеет смысл ограничивать.

<img src="assets/trees2.png" width="1000" align="center">

<center><i>Рис. 3. Обучение решающих деревьев в зависимости от максимального количества листьев.</i></center>

На практике чаще ограничивают не суммарное количество листьев, а максимальную глубину дерева, то есть максимальную длину пути от корня к листу. Можно также воспользоваться следующим критерием: прекращать рост дерева, если после добавления нового разделяющего правила функция потерь упала на величину, меньшую некоего порога. Однако такой подход тоже имеет свои проблемы.

На *рис. 4а* показана ситуация, когда добавление первого оптимального разделяющего правила не приведет к значимому снижению ошибки классификации, тогда как следующие два разделяющих правила ведут к радикальному снижению функции потерь. Более того, оптимальное значение порога для первого разделяющего правила сильно зависит от случайных факторов и может быть выбрано не равным нулю. На этом примере мы видим ситуацию "отложенного выигрыша", когда жадный алгоритм построения дерева может сработать не лучшим образом.

<img src="assets/trees3.jpg" width="400" align="center">

<center><i>Рис. 4. Первое решающее правило не даст выигрыша в точности.</i></center>

Обрезкой решающего дерева (pruning) называется процесс удаления из него отдельных ветвей, которые не приводят к существенному падению функции потерь. Имеел ли смысл сначала строить, а затем удалять ветви? Как мы увидели на *рис. 4*, при построении ветви мы не можем точно сказать, насколько сильно эта ветвь и дочерние к ней ветви помогут снизить функцию потерь. Это станет понятно только тогда, когда мы построим дочерние ветви, затем дочерние к дочерним и так далее. Если даже после этого функцию потерь на данной ветви не удалось сушественно снизить, тогда всю ветвь можно удалить, упростив дерево. См. также [Соколов, 2018]($Решающие деревья Е. А. Соколов$), раздел 5.

### Работа с категориальными признаками

До сих пор мы рассматривали работу только с количественными признаками. Если в обучающей выборке $N$ примеров, то в количественном признаке имеет смысл перебирать не более $N-1$ значений порога. Теперь рассмотрим категориальный признак с $K$ категориями. Можно разделить все категории на два подмножества, и отправить эти подмножества в разные ветви. Но количество возможных делений всех категорий на два подмножества растет экспоненциально с ростом количества категорий $K$, что делает невозможным полный перебор при большом $K$.

Есть два достаточно простых пути. Мы можем закодировать категориальный признак one-hot кодированием. Тогда если по этому признаку произойдет деление, то одна категория отправится в одву ветвь, все остальные в другую, то есть мы ищем только деления вида "одина категория против всех". Также мы можем оставить признаки в label-кодировании и рассматривать их как количественный признак, тогда в одну из ветвей отправятся все категории, индексы которых меньше определенного числа. Такие деления получаются менее осмысленными, но из этого напрямую не следует, что эффективность такого подхода будет ниже. Могут быть и другие подходы, например мы можем перебрать какое-то количество случайных делений всех категорий на два подмножества или упорядочить категории по среднему значению целевой переменной и искать разбиение как для количественного признака (см. [Соколов, 2018]($Решающие деревья Е. А. Соколов$), раздел 7).

На практике установлено, что для категориальных признаков небольшой размерности (т. е. с небольшим количеством категорий) лучше работает one-hot кодирование (см. [Prokhorenkova et al., 2017]($CatBoost: unbiased boosting with categorical features$)). Для признаков большой размерности можно тоже применять one-hot кодирование, хотя если сделать это в явном виде, то получившаяся матрица признаков будет занимать очень много места в памяти. Более эффективным подходом к работе с категориальными признаками большой размерности считается target-кодирование.

При target-кодировании мы заменяем каждую категорию некой статистикой (обычно средним значением) целевой переменной, рассчитанной по объектам данной категории. Например, если категориальной переменной является модель автомобиля, а целевым признаком является цена, то мы рассчитываем среднюю цену по каждой модели, и используем полученные данные вместо модели автомобиля.

Данный подход имеет два недостатка. Во-первых, представим, что в каждой категории только один объект. Тогда после target-кодирования признак будет содержать готовые ответы. Модели, обучаемой на этом датасете, будет достаточно извлекать ответ из этого признака, не используя никакие другие признаки. Очевидно, такой подход приведет к переобучению. Как вариант, можно использовать две обучающие выборки: на первой расссчитывать статистику по целевой переменной, а на второй с помощью этой статистики делать target-кодирование и обучать модель.

Вторая проблема в том, что категориальные признаки могут влиять на целевой признак не независимо друг от друга. Например, пусть мы имеем два категориальных признака в label-кодировани, принимающие значения 0 или 1: если они не равны друг другу, то целевой признак равен 1, в противном случае целевой признак равен 0. Если мы выполним target-кодирование, то эта информация будет полностью утеряна.

Несмотря на эти недостатки, target-кодирование и его различные варианты остается одним из самых эффективных способов работы с категориальными признаками высокой размерности. Авторы библиотеки CatBoost разработали метод упорядоченного target-кодирования, при котором на обучающих примерах задается некий порядок, и для каждого $i$-го примера статистика по целевой переменной рассчитывается только на основе примеров с индексами меньше $i$ (см. далее в разделе "CatBoost: несмещённый упорядоченный бустинг"). Обзор различных способов target-кодирования также можно найти в [Pargent et al., 2021]($Regularized target encoding outperforms traditional methods in supervised machine learning with high cardinality features$).

### Другие особенности решающих деревьев

Одним из преимуществ решающих деревьев является отсутствие необходимости нормализации данных и в целом почти полная нечувствительность к монотонным преобразованиям количественных признаков (например, логарифмированию).

Еще одним преимуществом является простота работы с пропущенными значениями. При вычислении суммарной функции потерь $(1)$ мы можем игнорировать те объекты, для которых значение признака не указано. Далее при построении дерева эти объекты можно отправять одновременно в обе ветви. При инференсе (то есть получении предсказаний с помощью уже построенного дерева) можно также отправлять объекты с неизвестным значением признака одновременно в обе ветви, и в обоих ветвях рассчитывать целевую переменную. После этого мы усредняем оба значения с весами, равными количеству обучающих примеров, прошедших через обе ветви (подробнее см. [Соколов, 2018]($Решающие деревья Е. А. Соколов$)).

## Ансамблирование решающих деревьев

Выше мы видели (*рис. 2, 3*), что одно решающее дерево имеет достаточно грубые границы между листьями, и при этом либо существенно недообучается (при малом количестве листьев), либо сильно переобучается (при большом количестве листьев). Эту проблему можно исправить, обучая сразу много разных решающих деревьев.

В целом ансамблированием называется комбинация нескольких моделей машинного обучения в одну модель. Ансамблирование решающих деревьев как правило осуществляется "одноуровнево", то есть при инференсе все деревья работают параллельно и независимо выдают ответ, а затем их предсказания складываются или усредняются. Процесс обучения при этом может выполняться параллельно (бэггинг) или последовательно (бустинг).

(**TODO стекинг**)

### Бэггинг

Алгоритм бэггинга достаточно прост: каждое дерево обучается на своей подвыборке данных, взятой из обучающей выборки. Подвыборка делается с возвращением, то есть один пример может быть выбран более одного раза. Если при этом в каждом дереве мы также будем использовать случайную подвыборку признаков, то получим алгоритм построения случайного леса, реализованный, например, в [sklearn](https://scikit-learn.org/stable/modules/ensemble.html#forest). Можно внести еще больше случайности в процесс ансамблирования, если процесс построения дерева также будет содержать элементы случайности (см. [Extremely Randomized Trees](https://scikit-learn.org/stable/modules/ensemble.html#extremely-randomized-trees)).

### Бустинг

Более распространенным методом ансаблирования решающих деревьев является бустинг. Бустинг - это способ построения ансамбля, в котором обучается много копий более слабой модели ("weak learner"), то есть такой модели, которая не может достичь высокой точности на обучающем датасете, переобучившись на нем. Как правило такой моделью является решающее дерево небольшой глубины. На каждом шаге новый weak learner концентрируется на исправлении ошибок, допущенных предыдущими weak learner'ами. В итоге предсказания всех weak learner'ов суммируются с определенными весами. Бустинг чем-то похож на бэггинг, но в бэггинге модели обучаются совершенно независимо и параллельно, а в бустинге последовательно, с оглядкой на предыдущие.

### Ранний вариант бустинга: AdaBoost

Итак, в бустинге каждый следующий weak learner стремится скорректировать предсказания предыдущих. Это можно сделать разными способами. Одной из первых эффективных реализаций бустинга был [AdaBoost]($Алгоритм AdaBoost$) - в нем каждый следующий weak learner фокусировал внимание на тех примерах, на которых предыдущие weak learner'ы дали неверные ответы. При этом он не знал, какие именно ответы даны предыдущими weak learner'ами - было лишь известно, что ответы неверны или неточны. Задачей нового weak learner'а было дать верные ответы преимущественно на этих примерах.

Заметим, что при этом не используется никакого валидационного датасета. Используется только обучающий датасет, на нем же оценивается точность предыдущих weak learner'ов. Это означает, что если очередной weak learner после обучения дал верные ответы на все примеры, то бустинг продолжить будет невозможно. Например, если в качестве weak learner'а мы используем решающее дерево неограниченной глубины, то так и произойдет. Нужно использовать решающие деревья небольшой глубины: weak learner должен быть действительно "слабым", не переобучаясь слишком сильно.

### Градиентный бустинг

В градиентном бустинге ([Friedman, 2001]($Greedy Function Approximation: A Gradient Boosting Machine$)) целевыми данными для следующего weak learner'а является *градиент (со знаком минус) функции потерь по предсказаниям предыдущих алгоритмов*. Таким образом следующий weak learner корректирует предсказания предыдущих.

Например, производная среднеквадратичной ошибки $(y - \hat{y})^2$ по $\hat{y}$ равна $2(\hat{y} - y)$, а значит предсказывая градиент weak learner как раз и предсказывает разность предсказания и правильного ответа. Но можно использовать и другие функции потерь.

Далее рассмотрим алгоритм градиентного бустинга более формально. Для дальнейшего изложения введем необходимые обозначения:

- **Обучающая выборка (датасет)** состоит из массива исходных данных $\textbf{X} = \{x_i\}_{i=1}^N$ и массива эталонных ответов $\textbf{y} = \{y_i\}_{i=1}^N$. Каждый $x_i$ представляет собой набор признаков - вектор фиксированной длины. В задаче классификации будем считать, что $\textbf{y}$ представлен в one-hot кодировании.

- **Обучаемый алгоритм** имеет параметры $\theta$, принимает на вход исходные данные и возвращает предсказания: $f(x_i, \theta) = \hat{y}_i$. Массив предсказаний алгоритма на всей обучающей выборке обозначим как $\hat{\mathbf{y}} = f(\textbf{x}, \theta)$. Количество параметров может быть как фиксированным (например, нейронная сеть), так и переменным (например, растущий случайный лес). Предсказание может быть, а может не быть дифференцируемым по параметрам $\theta$. Для *градиентного спуска* требуется дифференцируемость по параметрам, для *градиентного бустинга* - не требуется.

- **Функция потерь** принимает на вход эталонный ответ и предсказание и выдает число: $loss(y_i, \hat{y}_i)$. Мы хотим минимизировать сумму значений функции потерь по всей обучающей выборке: $sum\_loss(\textbf{y}, \hat{\mathbf{y}}) = \sum_{i=1}^{n} loss(y_i, \hat{y}_i)$. Как для градиентного спуска, так и для градиентного бустинга требуется дифференцируемость функции потерь по предсказаниям.

### Алгоритм градиентного бустинга

Пусть мы имеем обучающую выборку, обучаемый алгоритм (weak learner) и функцию потерь. В качестве исходного приближения выберем константу $C$ так, чтобы сумма $\sum_{i=1}^n loss(y_i, C)$ была минимальна. К начальному приближению мы будем прибавлять предсказания weak learner'ов. 
Заранее выберем число шагов $N$.

**for k = 0, ..., N-1:**

1. На $k$-м шаге мы уже обучили $k$ weak learner'ов. Мы получаем предсказания с помощью их взвешенной суммы для всех примеров из обучающего датасета:  
$\hat{y}_i = С + \sum\limits_{j=0}^k w_i f(x_i, \theta_j)$
2. Считаем производную функции потерь со знаком минус по каждому предсказанию:  
$r_i = -\cfrac{\partial}{\partial \hat{y}_i} loss(y_i, \hat{y}_i)$.  
Таким образом мы получаем информацию о том, как нам нужно изменить каждое предсказание, чтобы функция потерь уменьшилась (исходя из смысла понятия производной).
3. Обучаем новый weak learner предсказывать $r_i$ по $x_i$. Обозначим параметры нового weak learner'а за $\theta_k$.
4. Осталось выбрать вес для нового weak learner'а. Для этого получаем предсказания нового weak learner'а на всей обучающей выборке: $\hat{r}_i = f(x_i, \theta_k)$. Затем подбираем такой вес $w_k$, чтобы значение $sum\_loss(\mathbf{y}, \hat{\mathbf{y}} + w_k*\hat{\mathbf{r}})$ было минимально.

В итоге мы получаем ансамбль из $N$ weak learner'ов. Алгоритм инференса (то есть предсказания на произвольных данных) выглядит аналогично:

 $\hat{y} = С + \sum_{j=0}^N w_i f(x, \theta_j)$

### Регуляризация градиентного бустинга

Для того, чтобы ослабить переобучение градиентного бустинга, применяются следующие техники:

**Subsampling.** Мы обучаем каждый следующий weak learner не на всей обучающей выборке, а на случайной подвыборке. В этом случае градиентный бустинг называется стохастическим.

**Shrinkage.** После того, как мы расчитали вес нового weak learner'а $w_k$, мы умножаем его на некоторое число меньше 1. Таким образом мы укорачиваем шаг в направлении градиента.

**Регуляризация weak learner'а.** Например, в sklearn можно использовать параметр `max_features`, который определяет, сколько случайно выбранных признаков будет использоваться при поиске оптимального разбиения в каждом узле дерева.

### Особенности градиентного бустинга

В случае выбора функции потерь $MSE$ ее производная пропорциональна разности предсказания и верного ответа, поэтому градиентный бустинг сводится к тому, что каждый следующий weak learner пытается предсказать величину ошибки, допущенную предыдущим. Но при этом сам допускает ошибки, и их пытается предсказать уже следующий weak learner, и так далее.

Важно, чтобы weak learner не был способен слишком сильно переобучиться, иначе следующим weak learner'ам будет не на чем учиться. Поэтому в качестве weak learner'а обычно выбирают деревья небольшой глубины. Линейная регрессия не подходит в качестве weak learner'а, поскольку взвешенная сумма линейных моделей линейна, поэтому и весь ансамбль получится линейным.

Интересно, что если мы выберем в градиентном бустинге специальную "экспоненциальную" функцию потерь, то мы получаем алгоритм, эквивалентный AdaBoost. Конечно с вычистельной точки зрения эти алгоритмы получатся все равно разные, но было доказано, что они эквивалентны в плане получаемого результата. Таким образом, алгоритм AdaBoost эквивалентен частному случаю градиентного бустинга.

### Связь с градиентным спуском

Идея как градиентного спуска, так и градиентного бустинга состоит в том, что мы *рассчитываем градиент функции потерь по предсказаниям*, а затем ходим сдвинуть предсказания в направлении, противоположном градиенту, и таким образом сделать их более точными.

Но в градиентном спуске это достигается с помощью распространения градиента на веса и обновления весов, а в градиентном бустинге с помощью прибавления предсказаний нового weak learner'а, который аппроксимирует градиент со знаком минус. Таким образом, в градиентном спуске используется фиксированное число параметров, а в градиентном бустинге - переменное (каждый новый weak learner содержит новые параметры).

Иногда градиентный бустинг рассматривают как покоординатный градиентный спуск в пространстве функций.

## Современные алгоритмы бустинга над решающими деревьями

### XGBoost

Библиотека [XGBoost](https://github.com/dmlc/xgboost) ([Chen and Guestrin, 2016]($XGBoost: A Scalable Tree Boosting System$)) является вычислительно эффективной реализацией градиентного бустинга над решающими деревьями. Помимо оптимизированного программного кода, авторы предлагают различные улучшения алгоритма.

Рассмотрим для примера задачу регрессии и введем следующие обозначения:

- $\{x_i, y_i\}_{i=1}^N$ - обучающая выборка
- $K$ - количество деревьев в ансамбле
- $f_k: x \to y$ - k-e дерево ансамбля как функция
- $F: x \to y$ - весь ансамбль как функция
- ${loss}: y_{true} \times y_{pred} \to \mathbb{R}$ - выбранная пользователем основная функция потерь
- $T_k$ - количество листьев в $k$-м дереве ансамбля
- $w_k$ - вектор, составленный из выходных значений на всех листьях $k$-го дерева

В XGBoost ответы суммируются по всем деревьям ансамбля:

$F(x) = \sum\limits_{k=1}^K f_k(x)$

Суммарная функция потерь в XGBoost выглядит следующим образом:

${total\_loss} = \sum\limits_{i=1}^N {loss}(y_i, F(x_i)) + \gamma\sum\limits_{k=1}^K T_k + \frac{1}{2}\lambda\sum\limits_{k=1}^K \|w_k\|^2$

Здесь $\gamma$, $\lambda$ - гиперпараметры. Первое слагаемое - это основная функция потерь, второе слагаемое штрафует деревья за слишком большое количество листьев, третье слагаемое за слишком большие предсказания. Третье слагаемое является нетипичным в машинном обучении. Оно обеспечивает то, что каждое дерево вносит минимальный вклад в результат. Функция потерь используется при построении каждого следующего дерева, то есть функция потерь оптимизируется по параметрам лишь последнего дерева, не затрагивая предыдущие. Для минимизации ${total\_loss}$ используется метод второго порядка, то есть рассчитываются не только производные, но и вторые производные функции потерь по предсказаниям предыдущих деревьев (подробнее см. [Chen and Guestrin, 2016]($XGBoost: A Scalable Tree Boosting System$), раздел 2.2).

При поиске каждого нового разделяющего правила, для каждого признака перебираются не все возможные значения порога, а значения с определенным шагом. Для этого на признаке рассчитывается набор персентилей, используя статистику из обучающего датасета. Поиск оптимального порога выполняется только среди этих персентилей. Это позволяет существенно сократить время перебора и ускорить обучение. Для работы с пропущенными значениями в каждом решающем правиле определяется ветвь, в которую будут отправлены объекты с пропущенным значением данного признака.

Однако основной ценностью библиотеки XGBoost является эффективная программная реализация. За счет разных оптимизаций, таких как эффективная работа с пропущенными значениями, поиск порога только среди персентилей, оптимизация работа с кэшем и распределенное обучение, достигается выигрыш в десятки или даже сотни раз по сравнению с наивной реализацией.

### CatBoost: несмещённый упорядоченный бустинг

Библиотека [CatBoost](https://catboost.ai/) ([Prokhorenkova et al., 2017]($CatBoost: unbiased boosting with categorical features$), [Dorogush et al, 2018]($CatBoost: gradient boosting with categorical features support$)) - еще одна эффективная реализация градиентного бустинга над решающими деревьями. Основные особенности алгоритма следующие (не в порядке важности):

1. Использование решающих таблиц (разновидности решающих деревьев)
2. Упорядоченное target-кодирование на категориальных признаках высокой размерности
3. Бустинг с упорядочиванием обучающих примеров

**Упорядоченное target-кодирование**

Упорядоченное target-кодирование мы уже кратко рассматривали в разделе "Работа с категориальными признаками". Рассмотрим некий категориальный признак $C$. Все обучающие примеры случайным образом упорядочиваются. Пусть $i$-й пример имеет категорию $c_i$. Для данного примера рассчитывается выбранная статистика целевой переменной для всех примеров с индексами строго меньше $i$, имеющими ту же категорию $c_i$. Полученное значение является новым признаком, который используется вместо исходного признака $C$. О том, какие статистики можно использовать, см. документацию CatBoost ([1](https://catboost.ai/en/docs/concepts/algorithm-main-stages_cat-to-numberic), [2](https://catboost.ai/en/docs/references/training-parameters/ctr)).

В таком подходе есть одна проблема: статистика целевой переменной на первых примерах будет рассчитана слишком неточно (будет иметь высокую дисперсию). Поэтому в CatBoost target-кодирование выполняется несколько раз, каждый раз с новым случайным упорядочиванием обучающей выборки.

**Использование решающих таблиц**

Решающая таблица является частным случаем забывчивого решающего дерева (oblivious decision tree). В таком дереве все решающие правила одного уровня (то есть на одном и том же расстоянии от корня) проверяют один и тот же признак ([Kohavi, 1994]($Bottom-Up Induction of Oblivious Read-Once Decision Graphs$), [Rokach and Maimon, 2005]($Decision Trees Rokach Maimon$)). Забывчивые решающие деревья разрабатывались для задач с большим количеством нерелевантных признаков.

В варианте, реализованном в CatBoost, на каждом уровне решающего дерева используется не только общий признак, но и общий порог разделения (аналогично [Modrý and Ferov, 2016]($Enhancing LambdaMART Using Oblivious Trees$)). Благодаря этому порядок следования разделяющих правил становится не важен: можно переставить уровни дерева, и его функционирование от этого не изменится. Такое дерево более естественно представлять в виде таблицы, в которой решающее правило соответствует колонке, в которой может быть значение 0 или 1. Если в дереве $N$ уровней , то в таблице $N$ колонок и $2^N$ строк, содержащих все возможные сочетания значений колонок. Каждой строке сопоставляется константное значение целевого признака. Как можно видеть, такая модель имеет достаточно слабое сходство с обычным решающим деревом, хотя и строится по тем же правилам.

На *рис. 5* показана работа CatBoost на примере задачи регрессии $y(x) = x_2 - x_1$ (сравните с *рис. 2*).

<img src="assets/trees4.png" width="1000" align="center">

<center><i>Рис. 5. Приближение непрерывной функции одной или несколькими решающими таблицами с помощью CatBoost.</i></center>

Несколько решающих таблиц можно объединить в одну, поэтому весь ансамбль CatBoost можно представить в виде одной решающей таблицы, что видно из *рис. 5*. При этом не любая сложная решающая таблица может быть разложена в сумму многих простых таблиц.

**Упорядоченный бустинг**

Алгоритм бустинга имеет одну достаточно очевидную проблему. Выполняя шаг бустинга, мы хотим скорректировать предсказания алгоритма $F$, обучив новый алгоритм $f$, то есть алгоритм $f$ должен выявить и исправить систематические ошибки, допущенные алгоритмом $F$. При этом для обучения $f$ мы используем те же обучающие данные $X$, что использовали для обучения $F$. Это означает, что мы оцениваем работу алгоритма $F$ (точнее, производные функции потерь по его предсказаниям) на тех же данных, на которых этот алгоритм обучался. Однако алгоритм $F$ переобучился на данных $X$, и его работа на этих данных уже не показательна.

Рассмотрим эту проблему более формально. Обозначим на $P(x, y)$ порождающее распределение, из которого взята обучающая выборка. Совместное распределение $x, y$ в обучающей выборке можно обозначить за $P_{train}$ ([эмпирическое распределение](https://ru.wikipedia.org/wiki/%D0%92%D1%8B%D0%B1%D0%BE%D1%80%D0%BE%D1%87%D0%BD%D0%B0%D1%8F_%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F_%D1%80%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F)). Алгоритм $F$ моделирует условное распределение $P(y|x)$, алгоритм $f$ обучается моделировать условное распределение производных:

$P(g(x, y)|x) = P(\cfrac{\partial {Loss}(y, F(x))}{\partial F(x)}|x)$

Целью $f$ является моделирование $P(g(x, y)|x)$ для всего порождающего распределения, то есть при $x, y {\sim} P$, однако мы имеем выборку из $P(g(x, y)|x)$ только для обучающих данных, то есть при $x, y {\sim} P_{train}$. Проблема заключается в том, что распределение производных на обучающей выборке $P_{train}$ является искаженным относительно распределения производных на всем порождающем распределении $P$:

$g(x, y) | x, y {\sim} P \neq g(x, y) | x, y {\sim} P_{train}$

В качестве примера, если $F$ максимально переобучился на $P_{train}$, то $g(x, y) | x, y {\sim} P \equiv 0$. Из-за этого $f$ получается не оптимальным, так как обучается на выборке не из того распределения, которое в идеале должен моделировать. В идеале хотелось бы использовать для оценки градиента данные, на которых алгоритм $F$ не обучался, но тогда нам нужно использовать новые данные на каждом шаге бустинга, что невозможно при большом числе шагов.

Можно привести еще такой пример. Представим, что ученик $A$ обучается решать математические задачи из класса $P$ (например, упрощение дробей), имея выборку из этих задач $P_{train}$, и у нас есть также тестовая выборка задач $P_{test}$. Часть примеров из $P_{train}$ ученик $A$ не понял и просто запомнил, но не идеально. Если после обучения ученика $A$ дать ему задачи из $P_{test}$, которые он никогда не видел, то часть задач он решит, на на других сделает на них какие-то осмысленные ошибки. Если же дать ученику $A$ снова решать задачи из $P_{train}$, то он будет пытаться вспомнить решение, и сделает во-первых меньше ошибок, во-вторых уже ошибки другого рода (неправильно вспомнит и напишет что-то бессмысленное, что сойдется с ответом). Это означает, что распределение ошибок может быть совсем разным на обучающих и тестовых данных, по причине переобучения и так называемой утечки данных ([target leakage](https://en.wikipedia.org/wiki/Leakage_(machine_learning))).

Несмотря на то, что градиентный бустинг содержит такую проблему, он все равно хорошо работает на практике, поэтому на эту проблему долгое время закрывали глаза. Авторы библиотеки CatBoost нашли способ ее решения, названный упорядоченным бустингом ([Prokhorenkova et al., 2017]($CatBoost: unbiased boosting with categorical features$), раздел 4).